In [1]:
#importing the libraries
import numpy as np
import numpy.random as nprand
import math as math
import pandas as pd 

In [2]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#!python3 -m spacy download en
#!pip install -U gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sao107\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#!pip install gensim
#!pip install spacy

In [4]:
#!pip install pyLDAvis

In [5]:
import re
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
#Using Stop Words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
#read iris.csv
import pandas as pd
df = pd.read_csv('security.csv',encoding='utf-8')
df.head()

,postid,tags,title,question,answers,words,creation_date,Related
0,40471439,blockchain diffie-hellman encryption javascript,js diffi hellman ssl blockchain auth,tri figur secur qualiti ssl mimick without nee...,NaN,encrypt|scripting|diffie hellman|ssl|security|...,2016-11-07 17:52:22.793 UTC,Yes
1,32771909,cryptography jar jarsigner java smartcard,sign jar smartcard,use pkcs11 smartcard work jarsign sign jar fil...,ad -sigalg algortihm avail [sha256withrsa case...,crypt|signing|certificate|validate|private key...,2015-09-24 22:23:45.947 UTC,Yes
2,15574818,java jespa ntlm servlets spring-security,cannot creat session respons commit ie7 ie8,work applic allow user login use ntlm form log...,NaN,security|login|credential|password|validate|au...,2013-03-22 16:04:18.12 UTC,Yes
3,19042957,asp.net c# security,strong name assembl,thought learn strong name creat one exampl cac...,refer strong-nam assembl normal assembl includ...,security|signing|public key|signature|private key,2013-09-27 04:51:44.143 UTC,Yes
4,59694966,ssh digital-ocean cyberduck,eof read packet,connect via sftp digitalocean vps username/pas...,need updat password termin window *open digit ...,sftp|username|password|authentication|login,11/1/2020 13:43,Yes


In [8]:
# Convert to list
data = df.words.values.tolist()

# Remove Emails
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:4])

['encrypt|scripting|diffie '
 'hellman|ssl|security|authentication|certificate|authorise|public key',
 'crypt|signing|certificate|validate|private key|dsa|rsa|availability',
 'security|login|credential|password|validate|authentication',
 'security|signing|public key|signature|private key']


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(len(data_words))


#print(data_words[:10])

385


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[100]]])

['security', 'openid', 'oauth', 'login', 'authorise', 'authentication', 'validate', 'hack']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
#!python -m spacy download en_core_web_sm

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm",disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:10])

[['encrypt', 'scripting', 'security', 'authentication', 'certificate', 'authorise', 'public', 'key'], ['sign', 'certificate', 'validate', 'private', 'key'], ['security', 'login', 'credential', 'password', 'validate', 'authentication'], ['security', 'sign', 'public', 'key', 'signature', 'private', 'key'], ['sftp', 'username', 'password', 'authentication', 'login'], ['encrypt'], ['delegation', 'transport', 'security', 'privilege', 'authentication'], ['security', 'encrypt'], ['security', 'transport', 'certificate', 'trust', 'validate', 'insecure', 'bypass'], ['certificate', 'signature', 'hash', 'encode', 'validate']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [17]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('authentication', 1),
  ('authorise', 1),
  ('certificate', 1),
  ('encrypt', 1),
  ('key', 1),
  ('public', 1),
  ('scripting', 1),
  ('security', 1)]]

In [18]:
# Build hdp model
hdp_model = gensim.models.HdpModel(corpus, 
                                   id2word, 
                                   chunksize=10,
                                   random_state=42)



In [19]:
# Print the Keyword in the 10 topics
pprint(hdp_model.print_topics())
doc_hdp = hdp_model[corpus]

[(0,
  '0.041*sign + 0.038*virus + 0.034*penetration + 0.033*hmac + 0.028*tls + '
  '0.025*adversarial + 0.024*cookie + 0.024*pbkdf + 0.022*spam + 0.021*spnego'),
 (1,
  '0.049*access + 0.037*spoof + 0.033*sign + 0.029*signature + 0.029*firewall '
  '+ 0.028*password + 0.027*sanitise + 0.026*tls + 0.022*rbac + 0.022*tamper'),
 (2,
  '0.040*tunnel + 0.035*mitm + 0.033*test + 0.032*table + 0.031*crypt + '
  '0.028*key + 0.027*burp + 0.026*trust + 0.024*validate + 0.024*protect'),
 (3,
  '0.041*sandbox + 0.034*crc + 0.031*access + 0.026*penetration + 0.025*snif + '
  '0.025*login + 0.025*intruder + 0.024*sign + 0.022*cipher + 0.021*control'),
 (4,
  '0.058*injection + 0.040*leak + 0.032*credential + 0.031*salt + '
  '0.028*validate + 0.028*thread + 0.025*crypt + 0.024*authentication + '
  '0.022*forge + 0.021*bypass'),
 (5,
  '0.038*sanitise + 0.038*delegation + 0.034*sensitive + 0.031*private + '
  '0.030*danger + 0.026*certificate + 0.026*audit + 0.022*access + '
  '0.022*encrypt + 0.02

In [21]:
hdp_model[corpus[0]]

[(69, 0.4901064398717601), (81, 0.4003818836660858)]